In [1]:
import numpy as np
import pymc as pm

import xarray as xr

from pymc_marketing.clv.models.beta_geo import BetaGeoModel
from pymc_marketing.clv.distributions import continuous_contractual

from lifetimes.fitters.beta_geo_fitter import BetaGeoFitter

from scipy.special import expit, hyp2f1

In [2]:
rng = np.random.default_rng(34)

a_true = 0.8
b_true = 2.5
alpha_true = 3
r_true = 4

# Number of subjects
N = 500
# Subject level parameters
p = pm.draw(pm.Beta.dist(a_true, b_true, size=N), random_seed=rng)
lam = pm.draw(pm.Gamma.dist(r_true, alpha_true, size=N), random_seed=rng)

T = pm.draw(pm.DiscreteUniform.dist(lower=20, upper=40, size=N), random_seed=rng)

data = continuous_contractual.rng_fn(rng, lam, p, T, 0, size=N)

recency = data[..., 0]
frequency = data[..., 1]
alive = 1 - data[..., 2]

In [3]:
model = BetaGeoModel(
    customer_id=np.arange(N),
    frequency=frequency,
    recency=recency,
    T=T,
)

In [4]:
model.fit(chains=1);

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [a, b, alpha, r]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 11 seconds.
/home/ricardo/miniconda3/envs/pymmmc/lib/python3.10/site-packages/arviz/data/base.py:221: UserWarning: More chains (1000) than draws (1). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


In [5]:
a = xr.DataArray([10, 20, 30], coords={"customer_id": range(3)})
b = xr.DataArray([3, 4], coords={"a": [30, 40]})
c = (b + a)
c.transpose("customer_id", "a").mean(("customer_id", "a"))

<xarray.DataArray ()>
array(23.5)

In [7]:
model.expected_probability_alive(np.arange(N), frequency, recency, T)

<xarray.DataArray (chain: 1, draw: 1000, customer_id: 500)>
array([[[1.77178601e-04, 6.39336399e-05, 9.89599883e-05, ...,
         1.01606742e-05, 3.87580543e-07, 1.62659379e-05],
        [1.57042685e-04, 5.66366587e-05, 8.97643466e-05, ...,
         8.76911701e-06, 3.48630257e-07, 1.54153940e-05],
        [1.62031718e-04, 6.08710433e-05, 8.96099220e-05, ...,
         9.72377742e-06, 3.64083561e-07, 1.68103170e-05],
        ...,
        [4.35264438e-05, 2.07515429e-05, 2.78013366e-05, ...,
         3.00041766e-06, 1.35433826e-07, 1.23333238e-05],
        [2.24439083e-04, 7.48667922e-05, 1.21583909e-04, ...,
         1.16158223e-05, 4.39413252e-07, 1.68733060e-05],
        [2.15888835e-04, 7.73232871e-05, 1.15060188e-04, ...,
         1.27732285e-05, 4.52420617e-07, 1.74784424e-05]]])
Coordinates:
  * customer_id  (customer_id) int64 0 1 2 3 4 5 6 ... 494 495 496 497 498 499
  * chain        (chain) int64 0
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999

In [8]:
def true_expected_number_of_purchases(t):
    left_term = (a_true + b_true - 1) / (a_true - 1)
    right_term = 1 - (alpha_true / (alpha_true + t)) ** r_true * hyp2f1(
        r_true, b_true, a_true + b_true - 1, t / (alpha_true + t)
    )
    return left_term * right_term

In [9]:
t = np.array(range(20, 40, 2))
true_expected_number_of_purchases(t)

array([ 7.83353342,  8.18144426,  8.50640294,  8.81155417,  9.0994203 ,
        9.37205729,  9.63116452,  9.8781637 , 10.11425711, 10.34047107])